In [18]:
# !pip install -U sentence-transformers

# from sentence_transformers import SentenceTransformer
# sentences = ["This is an example sentence", "Each sentence is converted"]

# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
# embeddings = model.encode(sentences)
# print(embeddings)


from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences1 = ['문과생도 중고등학생도 직장인도 프로그래밍에 눈뜨게 만든 바로 그 책이 전면 개정판으로 새로 태어났다!', 
             '2016년 《Do it! 점프 투 파이썬》으로 출간되었던 이 책은 약 4년 동안의 피드백을 반영하여 초보자가 더 빠르게 입문하고, 더 깊이 있게 공부할 수 있도록 개정되었다.',
             '특히 ‘나 혼자 코딩’과 ‘코딩 면허 시험 20제’ 등 독자의 학습 흐름에 맞게 문제를 보강한 점이 눈에 띈다.',
             '실습량도 두 배로 늘었다.',
              '4년 동안 압도적 1위!', 
              '위키독스 누적 방문 200만!'] 
            #   '수많은 대학 및 학원의 교재 채택 등!', '검증은 이미 끝났다.', 
            #  '코딩을 처음 배우는 중고등학생부터 나만의 경쟁력이 필요한 문과생, 데이터 분석과 인공지능/머신러닝으로 커리어를 뻗어 나가고 싶은 직장인까지!',
            #  '프로그래밍의 세계에 풍덩 빠져 보자.']

sentences2 = ['『혼자 공부하는 파이썬』이 더욱 흥미있고 알찬 내용으로 개정되었다.',
              '프로그래밍이 정말 처음인 입문자도 따라갈 수 있는 친절한 설명과 단계별 학습은 그대로!' ,
              '혼자 공부하더라도 체계적으로 계획을 세워 학습할 수 있도록 ‘혼공 계획표’를 새롭게 추가했다.', 
              '또한 입문자가 자주 물어보는 질문과 오류 해결 방법을 적재적소에 배치하여 예상치 못한 문제에 부딪혀도 좌절하지 않고 끝까지 완독할 수 있도록 도와준다.',
              '단순한 문법 암기와 코딩 따라하기에 지쳤다면, 새로운 혼공파와 함께 ‘누적 예제’와 ‘도전 문제’로 프로그래밍의 신세계를 경험해 보자!',
              '배운 내용을 씹고 뜯고 맛보고 즐기다 보면 응용력은 물론 알고리즘 사고력까지 키워 코딩 실력이 쑥쑥 성장할 것이다.']

sentences_list = [sentences1,sentences2]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Tokenize sentences
encoded_input1 = tokenizer(sentences1, padding=True, truncation=True, return_tensors='pt')
encoded_input2 = tokenizer(sentences2, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output1 = model(**encoded_input1)
    model_output2 = model(**encoded_input2)

# Perform pooling. In this case, max pooling.
sentence_embeddings1 = mean_pooling(model_output1, encoded_input1['attention_mask'])
sentence_embeddings2 = mean_pooling(model_output2, encoded_input2['attention_mask'])


print("Sentence embeddings:")
print(sentence_embeddings1)
print(sentence_embeddings2)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Sentence embeddings:
tensor([[ 0.0081,  0.1814, -0.0161,  ...,  0.0409, -0.0631, -0.1327],
        [-0.0423,  0.0800, -0.0111,  ...,  0.0533,  0.0478, -0.0486],
        [-0.0736,  0.0722, -0.0127,  ...,  0.0165, -0.0259, -0.0735],
        [ 0.0414, -0.0219, -0.0199,  ..., -0.0053,  0.0541, -0.0447],
        [-0.0653,  0.1031, -0.0192,  ...,  0.0791,  0.0363, -0.1438],
        [ 0.0511,  0.1320, -0.0190,  ...,  0.0949,  0.0179, -0.1092]])
tensor([[ 0.0259,  0.1227, -0.0138,  ..., -0.0535, -0.0571, -0.0007],
        [-0.0673,  0.0668, -0.0143,  ...,  0.0540, -0.0257, -0.0945],
        [-0.0762,  0.0754, -0.0104,  ..., -0.0009, -0.1301, -0.0973],
        [ 0.0338,  0.0480, -0.0129,  ...,  0.0269,  0.0272, -0.0463],
        [ 0.0576,  0.1811, -0.0141,  ...,  0.1309, -0.1167, -0.1311],
        [-0.0355,  0.2336, -0.0112,  ...,  0.0591,  0.0213, -0.1165]])


In [19]:
torch.cosine_similarity(sentence_embeddings1.reshape(1,-1),sentence_embeddings2.reshape(1,-1))

tensor([0.3542])

In [24]:
cos_sim = list(torch.cosine_similarity(sentence_embeddings1,sentence_embeddings2))
cos_sim

[tensor(0.6309),
 tensor(0.4219),
 tensor(0.5596),
 tensor(0.1666),
 tensor(0.1892),
 tensor(0.1916)]

In [25]:
cos_sim = sum(cos_sim)/len(cos_sim)
cos_sim

tensor(0.3600)